In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("21"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Keypad Conundrum

[Open Website](https://adventofcode.com/2024/day/21)

In [ ]:
from collections import deque
from common.matrix import Direction, Matrix, MatrixNavigator


# helper functions
class KeyPad:
    def __init__(self, input):
        self.matrix = Matrix(input)
        for x, y, value in self.matrix:
            if value == 'A':
                self.pointer = MatrixNavigator(self.matrix, x, y)
                break

    def move_to(self, value: str) -> list:
        # find end
        end = None
        for x, y, v in self.matrix:
            if v == value:
                end = (x, y)
                break

        if end is None:
            raise ValueError(f"Value {value} not found in matrix")

        start = self.pointer.get_position()
        path = find_shortest_path(self.matrix, start, end, {})
        self.pointer.set_position(end[0], end[1])

        return path

    def input_value(self, string: str, debug: bool) -> str:
        # find steps
        presses = []
        for next in list(string):
            if debug:
                print(f"Press {next} next")
            path = self.move_to(next)
            for step in path:
                presses.append(direction_string(step))
            presses.append("A")

        return presses


def direction_string(direction: Direction) -> str:
    if direction == Direction.UP:
        return "^"
    elif direction == Direction.DOWN:
        return "v"
    elif direction == Direction.LEFT:
        return "<"
    elif direction == Direction.RIGHT:
        return ">"
    else:
        return "A"


directions = [
    Direction.UP,
    Direction.DOWN,
    Direction.LEFT,
    Direction.RIGHT
]


direction_cost_cache = {}
def cost_of_direction(last_key: Direction, next_key: Direction) -> int:
    key = (last_key, next_key)
    if key in direction_cost_cache:
        return direction_cost_cache[key]
    
    couple = (last_key, next_key)
    
    if last_key is None:
        if next_key == Direction.LEFT:
            cost = 3
        elif next_key == Direction.DOWN:
            cost = 2
        else:
            cost = 1
    else:
        if last_key == next_key:
            cost = 0
        elif Direction.UP in couple and Direction.DOWN in couple:
            cost = 1
        else:
            cost = 2
    
    return cost
        
    

def find_shortest_path(matrix: Matrix, start: tuple, end: tuple, cache: dict) -> list:
    key = (start, end)
    if start in cache:
        return cache[key]

    queue = deque([(start, 0, [])])
    visited = set()
    visited.add(start)

    while queue:
        current_point, cost, path = queue.popleft()
        if current_point == end:
            cache[key] = path
            return path

        for direction in directions:
            pointer = MatrixNavigator(matrix, current_point[0], current_point[1])
            last_direction = path[-1] if path else None
            new_cost = cost_of_direction(last_direction, direction)
            ok = pointer.move(direction)
            if ok:
                new_path = path + [direction]
                queue.append((pointer.get_position(), cost + new_cost, new_path))

    return []  # Return an empty list if no path is found


def new_number_pad() -> KeyPad:
    input = [
        ['7', '8', '9'],
        ['4', '5', '6'],
        ['1', '2', '3'],
        [' ', '0', 'A']
    ]
    return KeyPad(input)


def new_direction_pad() -> KeyPad:
    input = [
        [' ', '^', 'A'],
        ['<', 'v', '>']
    ]
    return KeyPad(input)

# <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
# v<<A>>^A<A>AvA<^AA>A<vAAA>^A
# <A^A>^^AvvvA
# 029A

def handle_line(line: str, debug: bool) -> int:
    # find multiplier, get digits in string
    multiplier = int("".join([char for char in line if char.isdigit()]))
    
    # find first key presses
    num_pad = new_number_pad()
    result = num_pad.input_value(input, debug)
    if debug:
        print(result)
        print(multiplier)

# test case (part 1)
def part_1(inputs: list, debug: bool) -> int:
    for input in inputs:
        handle_line(input, debug)
    return 0


result = part_1(["029A"], True)
display(result)
# assert result == 126384

TypeError: 'method' object is not iterable

In [ ]:
# real case (part 1)
result = part_1(puzzle.input(), False)
display(result)

In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        display(lines)
    return 0


result = part_2(puzzle.example(0), True)
display(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()